In [1]:
import os
import time
import math
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import einops
import h5py
from tqdm import tqdm

import matplotlib.pyplot as plt

In [2]:
# device = 'cuda:1'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
torch.set_default_tensor_type('torch.cuda.DoubleTensor')

In [3]:
# path_dataset = 'Univariate_arff/'
# path_dataset_csv = path_dataset + 'dataset.csv'
# dataset85 = 'dataset85.txt'

# dataset_names = list(pd.read_csv(path_dataset_csv, encoding='ISO-8859-1', header=None)[:85][1].values)
# with open(dataset85, 'w') as f:
#     f.write('\n'.join(dataset_names))

In [4]:
def load_dataset(path_dataset, dataset_name):

    training_data = np.loadtxt(f"{path_dataset}/{dataset_name}/{dataset_name}_TRAIN.txt")
    Y_training_, X_training = training_data[:, 0].astype(np.int32), training_data[:, 1:]

    test_data = np.loadtxt(f"{path_dataset}/{dataset_name}/{dataset_name}_TEST.txt")
    Y_test_, X_test = test_data[:, 0].astype(np.int32), test_data[:, 1:]

    classes_old = np.unique(Y_training_)
    n_classes = classes_old.shape[0]
    
    Y_training = np.zeros(Y_training_.shape[0]).astype(np.int32)
    Y_test = np.zeros(Y_test_.shape[0]).astype(np.int32)
    for i, c in enumerate(classes_old):
        Y_training[Y_training_ == c] = i
        Y_test[Y_test_ == c] = i

    return X_training, Y_training, X_test, Y_test, n_classes, classes_old

In [6]:
class RandomFeatures(nn.Module):
    kernel_length = torch.tensor([7, 9, 11], dtype=torch.float)
    weight = None
    bias = None
    dilation = None
    padding = None
    
    def __init__(self, n_kernels, ts_length, random_seed=0):
        super(RandomFeatures, self).__init__()
        
        self.n_kernels = n_kernels
        self.ts_length = ts_length
        self.random_seed = random_seed * n_kernels
        self.create_kernels(self.random_seed)
        
    def create_kernels(self, random_seed=0):
        torch.manual_seed(random_seed)
        kernel_length_indices = torch.multinomial(self.kernel_length, self.n_kernels, replacement=True)
        kernel_lengths = self.kernel_length[kernel_length_indices].to(torch.long)
        weight_distributions_ = [torch.normal(mean=0, std=1, size=(kernel_length,), dtype=torch.double) for kernel_length in kernel_lengths]
        weight_distributions = [weight_distribution - weight_distribution.mean() for weight_distribution in weight_distributions_]
        self.weight = [weight_distribution.unsqueeze(0).unsqueeze(0) for weight_distribution in weight_distributions]
        
        self.bias = torch.rand(size=(self.n_kernels,), dtype=torch.double).reshape(-1,1) * 2 - 1
        A = math.log(self.ts_length-1) - torch.log(kernel_lengths-1)
        s = torch.rand(size=(self.n_kernels,)) * A
        self.dilation = torch.floor(2**s)
        self.padding = ((self.dilation * (kernel_lengths-1) / 2) * torch.randint(2, size=(self.n_kernels,)))
    
    
    def forward(self, x):
        batch_size = x.size(0)
        x = einops.rearrange(x, 'b t -> b 1 t')
        features = torch.empty(size=(batch_size, self.n_kernels, 2))
        
        for i in range(self.n_kernels):
            ts_convolved = nn.functional.conv1d(
                x, weight=self.weight[i], bias=self.bias[i], 
                padding=int(self.padding[i].item()), dilation=int(self.dilation[i].item())
            )
            ts_convolved = ts_convolved.squeeze(1)

            ts_convolved_max = torch.max(ts_convolved, dim=1).values
            features[:,i,0] = ts_convolved_max

            ts_convolved_ppv = torch.mean((ts_convolved > 0).to(torch.float), dim=1)
            features[:,i,1] = ts_convolved_ppv
        
        features = einops.rearrange(features, 'b s f -> b (s f)')
        return features

In [7]:
class LogisticRegression(nn.Module):
    def __init__(self, n_features, n_classes, mc_dropout=0.3, random_seed=0):
        super(LogisticRegression, self).__init__()
        self.n_features = n_features
        self.n_classes = n_classes
        self.mc_dropout = mc_dropout
        self.random_seed = random_seed
        
        torch.manual_seed(self.random_seed)
        self.linear = nn.Linear(n_features, n_classes)
        self.input_dropout = nn.Dropout(p=self.mc_dropout)
    
    def update_dropout(self, mc_dropout=0.3):
        self.input_dropout = nn.Dropout(p=mc_dropout)
    
    def forward(self, x, use_dropout=False):
        if use_dropout:
            x = self.input_dropout(x)
        logits = self.linear(x)
        return logits

In [8]:
class Trainer:
    def __init__(self, model, lr=1e-3):
        self.model = model
        self.lr = lr
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr, weight_decay=1)
        self.criterion = nn.CrossEntropyLoss()
        self.logsoftmax = torch.nn.LogSoftmax(dim=1)
        self.train_loss = []

    def fit(self, x, y, nepoch=1, lr=1e-3, tol=None, counter_thr = 10):
        optimizer = torch.optim.Adam(
            [{'params':param, 'lr':lr} for param in self.model.parameters()]
        )
        counter = 0
        for epoch in range(nepoch):
            self.model.train()
            optimizer.zero_grad()
            logits = self.model(x)
            loss = self.criterion(logits, y)
            loss.backward()
            optimizer.step()
            self.train_loss.append(loss.detach().cpu().numpy().item())
            if tol is not None and epoch > 1:
                if abs(self.train_loss[-1] - self.train_loss[-2]) <= tol:
                    counter += 1
                else:
                    counter = 0
            if counter >= counter_thr:
                break
            
    def predict_logits(self, x):
        self.model.eval()
        logits_unscaled = self.model(x)
        logits = self.logsoftmax(logits_unscaled)
        return logits
    
    def predict_mc_logits(self, x, n_samples_mc=100):
        batch_size = x.size(0)
        self.model.train()
        logits_unscaled = torch.empty(size=(batch_size, self.model.n_classes, n_samples_mc))
        for i in range(n_samples_mc):
            logits_unscaled_ = self.model(x, use_dropout=True)
            logits_unscaled[:,:,i] = logits_unscaled_.detach().clone()
        logits_sum = self.logsoftmax(logits_unscaled)
        logits = torch.logsumexp(logits_sum, dim=2) - math.log(n_samples_mc)
        return logits

In [9]:
def accuracy(logits, targets):
    # probs = np.exp(logits.detach().cpu().numpy())
    probs = torch.exp(logits)
    correspondence = (torch.argmax(probs, axis=1) == targets).to(torch.float)
    return correspondence.mean().detach().cpu().numpy().item()

def negative_loglikelihood(logits, targets):
    nll = nn.NLLLoss()
    return nll(logits, targets).detach().cpu().numpy().item()

def brier_score(logits, targets, n_classes):
    probs = torch.exp(logits)
    targets_one_hot = nn.functional.one_hot(targets, num_classes=n_classes)
    square_errors = (probs - targets_one_hot)**2
    return square_errors.mean().detach().cpu().numpy().item()

def expected_calibration_error(logits, targets, n_bins=10):
    bins_low = torch.arange(n_bins) / n_bins
    bins_high = (torch.arange(n_bins) + 1) / n_bins

    probs = torch.exp(logits)
    p_hat = torch.max(probs, dim=1).values
    targets_hat = torch.argmax(probs, dim=1)
    accuracy = (targets_hat == targets)
    
    calibration_errors = []
    for i in range(n_bins):
        mask = ((p_hat >= bins_low[i] if i==0 else p_hat > bins_low[i]) * (p_hat <= bins_high[i]))
        if torch.any(mask):
            bin_size = torch.sum(mask.to(torch.float)).item()
            p_hat_masked = torch.mean(p_hat[mask].to(torch.float)).item()
            accuracy_masked = torch.mean(accuracy[mask].to(torch.float)).item()
            calibration_error = abs(accuracy_masked - p_hat_masked)
            calibration_errors.append(calibration_error * bin_size / targets.size(0))
    return np.sum(calibration_errors)

### Single Trial

In [10]:
class SingleTrialEstimate():
    def __init__(self, n_kernels, ts_length, n_classes, n_ece_bins, random_seed=0):
        self.n_ece_bins = n_ece_bins
        self.n_classes = n_classes
        self.random_seed = random_seed
        n_features = 2 * n_kernels
        self.rf = RandomFeatures(n_kernels, ts_length, random_seed)
        self.lr = LogisticRegression(n_features, n_classes, random_seed=random_seed)
        self.fitted = False
        self.trainer = None
        
    def fit(self, X, Y, nepoch=5000, lr=1e-4, tol=1e-4, counter_thr=300):
        X = torch.tensor(X, dtype=torch.double)
        Y = torch.tensor(Y, dtype=torch.long)
        
        features = self.rf(X).detach().clone()
        self.trainer = Trainer(self.lr)
        self.trainer.fit(features, Y, nepoch=nepoch, lr=lr, tol=tol, counter_thr=counter_thr)
        self.fitted = True
        
    def score(self, X, Y):
        assert self.fitted, 'model should be fitted'
        X = torch.tensor(X, dtype=torch.double)
        Y = torch.tensor(Y, dtype=torch.long)
        
        features = self.rf(X).detach().clone()
        logits = self.trainer.predict_logits(features)
        results = {}
        results['acc'] = accuracy(logits, Y)
        results['nll'] = negative_loglikelihood(logits, Y)
        results['bs'] = brier_score(logits, Y, self.n_classes)
        results['ece'] = expected_calibration_error(logits, Y, self.n_ece_bins)
        return results

In [11]:
class MCDropoutEstimate(SingleTrialEstimate):
    def __init__(self, n_kernels, ts_length, n_classes, n_ece_bins, random_seed=0,
                 mc_dropout=0.01, n_samples_mc=100):
        super(MCDropoutEstimate, self).__init__(n_kernels, ts_length, n_classes, n_ece_bins, random_seed)
        self.mc_dropout = mc_dropout
        self.n_samples_mc = n_samples_mc
        
    def score(self, X, Y):
        assert self.fitted, 'model should be fitted'
        X = torch.tensor(X, dtype=torch.double)
        Y = torch.tensor(Y, dtype=torch.long)
        
        features = self.rf(X).detach().clone()
        self.trainer.model.update_dropout(self.mc_dropout)
        logits = self.trainer.predict_mc_logits(features, self.n_samples_mc)
        results = {}
        results['acc'] = accuracy(logits, Y)
        results['nll'] = negative_loglikelihood(logits, Y)
        results['bs'] = brier_score(logits, Y, self.n_classes)
        results['ece'] = expected_calibration_error(logits, Y, self.n_ece_bins)
        return results

In [12]:
class EnsembleEstimate():
    def __init__(self, n_kernels, ts_length, n_classes, n_ece_bins, random_seed=0, 
                 n_random_features=10):
        self.n_ece_bins = n_ece_bins
        self.n_classes = n_classes
        self.random_seed = random_seed + 1
        self.n_random_features = n_random_features
        n_features = 2 * n_kernels
        self.rf = [RandomFeatures(n_kernels, ts_length, random_seed+i) for i in range(self.n_random_features)]
        self.lr = [LogisticRegression(n_features, n_classes, random_seed=random_seed+i) for i in range(self.n_random_features)]
        self.fitted = False
        self.trainer = None
        
    def fit(self, X, Y, nepoch=5000, lr=1e-4, tol=1e-4, counter_thr=300):
        X = torch.tensor(X, dtype=torch.double)
        Y = torch.tensor(Y, dtype=torch.long)
        self.trainer = [Trainer(lr) for lr in self.lr]
        for i in range(self.n_random_features):
            features = self.rf[i](X).detach().clone()
            self.trainer[i].fit(features, Y, nepoch=nepoch, lr=lr, tol=tol, counter_thr=counter_thr)
        self.fitted = True
        
    def score(self, X, Y):
        assert self.fitted, 'model should be fitted'
        X = torch.tensor(X, dtype=torch.double)
        Y = torch.tensor(Y, dtype=torch.long)
        
        logits_ensemble = torch.empty(size=(X.shape[0], self.n_classes, self.n_random_features))
        for i in range(self.n_random_features):
            features = self.rf[i](X).detach().clone()
            logits_ = self.trainer[i].predict_logits(features)
            logits_ensemble[:,:,i] = logits_
        logits_ensemble = torch.nn.LogSoftmax(dim=1)(logits_ensemble)
        logits = torch.logsumexp(logits_ensemble, dim=2) - math.log(self.n_random_features)
            
        results = {}
        results['acc'] = accuracy(logits, Y)
        results['nll'] = negative_loglikelihood(logits, Y)
        results['bs'] = brier_score(logits, Y, self.n_classes)
        results['ece'] = expected_calibration_error(logits, Y, self.n_ece_bins)
        return results

In [13]:
class FFTEstimate(SingleTrialEstimate):
    def __init__(self, n_kernels, ts_length, n_classes, n_ece_bins, random_seed=0,
                 n_samples=10, style='full'):
        super(FFTEstimate, self).__init__(n_kernels, ts_length, n_classes, n_ece_bins, random_seed)
        self.n_samples = n_samples
        self.style = style

    def score(self, X, Y):
        assert self.fitted, 'model should be fitted'
        np.random.seed(self.random_seed)
        
        X = torch.tensor(X, dtype=torch.double)
        Y = torch.tensor(Y, dtype=torch.long)
        
        n_test = X.size(0)

        X_fft = torch.fft.fft(X, dim=1)
        X_fft_abs = np.abs(X_fft.detach().cpu().numpy())
        X_fft_angle = np.angle(X_fft.detach().cpu().numpy())

        X_fft_angle_sampled = []
        l = (X_fft_angle.shape[1]-1)//2
        
        if self.style == 'odd':
            amp = X_fft_abs[:,1:l+1][:,::2]
        elif self.style == 'even':
            amp = X_fft_abs[:,1:l+1][:,1::2]
        elif self.style == 'full':
            amp = X_fft_abs[:,1:l+1]
        elif self.style == 'unif':
            amp = np.ones_like(X_fft_abs)[:,1:l+1]
        p = amp / np.sum(amp, axis=1, keepdims=True)

        for i in range(n_samples):
            X_fft_angle_ = np.copy(X_fft_angle)
            for j in range(n_test):
     
                if self.style == 'odd':
                    index = np.random.choice(np.arange(1, l+1)[::2], size=1, p=p[j])
                elif self.style == 'even':
                    index = np.random.choice(np.arange(1, l+1)[1::2], size=1, p=p[j])
                elif self.style == 'full':
                    index = np.random.choice(np.arange(1, l+1), size=1, p=p[j])
                elif self.style == 'unif':
                    index = np.random.choice(np.arange(1, l+1), size=1, p=p[j])
                phase = (np.random.rand(1) - 0.5) * 2 * np.pi
                X_fft_angle_[j,index] = phase
                X_fft_angle_[j,-index] = - phase
            X_fft_angle_sampled.append(X_fft_angle_)

        X_fft_angle_sampled = np.stack(X_fft_angle_sampled)
        X_fft_sampled = torch.tensor(X_fft_abs*np.exp(1j*X_fft_angle_sampled))
        X_sampled = torch.real(torch.fft.ifft(X_fft_sampled, dim=2))
        
        
        logits_sampled = torch.empty(n_test, self.n_classes, self.n_samples)
        for i in range(self.n_samples):
            X_ = X_sampled[i]
            features_ = self.rf(X_).detach().clone()
            logits_ = self.trainer.predict_logits(features_).detach().clone()
            logits_sampled[:,:,i] = logits_

        logits_sampled = torch.nn.LogSoftmax(dim=1)(logits_sampled)
        logits = torch.logsumexp(logits_sampled, dim=2) - math.log(self.n_samples)
        
        results = {}
        results['acc'] = accuracy(logits, Y)
        results['nll'] = negative_loglikelihood(logits, Y)
        results['bs'] = brier_score(logits, Y, self.n_classes)
        results['ece'] = expected_calibration_error(logits, Y, self.n_ece_bins)
        return results

In [6]:
path_dataset = 'Univariate_arff/'
dataset85 = 'dataset85.txt'
with open(dataset85, 'r') as f:
    dataset_names = list(map(str.strip, f.readlines()))

n_datasets = len(dataset_names)

n_kernels = 10000
random_seed = 0
n_ece_bins = 10

n_samples = 100
mc_dropout = 0.01
n_samples_mc = 1000
n_random_features = 10
n_samples = 100

In [7]:
dataset_name = 'ElectricDevices'
X_training, Y_training, X_test, Y_test, n_classes, classes_old = load_dataset(path_dataset, dataset_name)

In [17]:
dataset_name = dataset_names[4]
X_training, Y_training, X_test, Y_test, n_classes, classes_old = load_dataset(path_dataset, dataset_name)
ts_length = X_training.shape[1]

estimators_names = ['ste', 'mcde', 'ee', 'ffteO', 'ffteE', 'ffteF', 'ffteU']

for estimators_name in estimators_names:
    if estimators_name == 'ste':
        est = SingleTrialEstimate(n_kernels, ts_length, n_classes, n_ece_bins, random_seed)
    elif estimators_name == 'mcde':
        est = MCDropoutEstimate(n_kernels, ts_length, n_classes, n_ece_bins, random_seed, mc_dropout, n_samples_mc)
    elif estimators_name == 'ee':
        est = EnsembleEstimate(n_kernels, ts_length, n_classes, n_ece_bins, random_seed, n_random_features)
    elif estimators_name == 'ffteO':
        est = FFTEstimate(n_kernels, ts_length, n_classes, n_ece_bins, random_seed, n_samples, 'odd')
    elif estimators_name == 'ffteE':
        est = FFTEstimate(n_kernels, ts_length, n_classes, n_ece_bins, random_seed, n_samples, 'even')
    elif estimators_name == 'ffteF':
        est = FFTEstimate(n_kernels, ts_length, n_classes, n_ece_bins, random_seed, n_samples, 'full')
    elif estimators_name == 'ffteU':
        est = FFTEstimate(n_kernels, ts_length, n_classes, n_ece_bins, random_seed, n_samples, 'unif')

    t1 = time.time()
    path_save = 'results/' + dataset_name + '_' + estimators_name + '.h5'
    est.fit(X_training, Y_training)
    result = est.score(X_test, Y_test)
    print((time.time() - t1)/60, estimators_name, result)
    
    # with h5py.File(path_save, 'w') as file:
    #     for k, v in result.items():
    #         file.attrs[k] = v

0.06665996710459392 ste {'acc': 0.9000000357627869, 'nll': 0.45585725154843065, 'bs': 0.10424116029228159, 'ece': 0.12803682088851928}
0.07454656759897868 mcde {'acc': 0.9000000357627869, 'nll': 0.45497639846637783, 'bs': 0.10437044272640786, 'ece': 0.12827165126800538}
0.6264220317204793 ee {'acc': 0.9000000357627869, 'nll': 0.4515085405461571, 'bs': 0.1053959328434176, 'ece': 0.14237320125102998}
2.660733699798584 ffteO {'acc': 0.9000000357627869, 'nll': 0.44926954472045366, 'bs': 0.12198390299057321, 'ece': 0.208628910779953}
2.6010375658671063 ffteE {'acc': 0.9000000357627869, 'nll': 0.49593707317142516, 'bs': 0.14892690024992222, 'ece': 0.2514934420585632}
2.6497342308362324 ffteF {'acc': 0.9000000357627869, 'nll': 0.4728118235254476, 'bs': 0.13161422635499892, 'ece': 0.22857072353363037}
2.6488574584325155 ffteU {'acc': 0.9000000357627869, 'nll': 0.452806532166381, 'bs': 0.10393166652685062, 'ece': 0.12664009928703307}


In [ ]:
# path_dataset = 'Univariate_arff/'
# path_dataset_csv = path_dataset + 'dataset.csv'
# dataset85 = 'dataset85.txt'

# dataset_names = list(pd.read_csv(path_dataset_csv, encoding='ISO-8859-1', header=None)[:85][1].values)
# with open(dataset85, 'w') as f:
#     f.write('\n'.join(dataset_names))